In [ ]:
%pip install pandas
%pip install pyspark==3.5.0
%pip install -q findspark
%pip install py4j
%pip install pyarrow
%pip install numpy

In [3]:
# for local operations
import os
os.environ['JAVA_HOME'] = "C:\\Program Files\\Java\\jdk-11"

In [4]:
import pandas as pd
import findspark
findspark.init()
findspark.find()
import pyspark

In [5]:
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]")  \
    .getOrCreate()
spark

Tip: In Google Colab, it is possible to read the entire folder content just by specifying its name, however, this doesn't work locally - you have to add `*` at the end of the path to get all the files or specify a single file. Another difference is that reading the whole dataset is much quickier on Colab (around a minute) than on the local machine. 

In [6]:
# rdd_csv = sc.wholeTextFiles('project_data/project/raw_time_series/csv')
# rdd_csv.collect()

rdd_json = sc.wholeTextFiles('project_data/project/customer_tariff/json/part-00000-44d37ba9-fccd-40f4-b939-d8b47af99c95-c000.json')
rdd_json.collect()

# rdd_parquet = sc.wholeTextFiles('project_data/project/raw_time_series/parquet')
# rdd_parquet.collect()

[('file:/c:/Users/liviu/Documents/fiipractic/big-data/project/project_data/project/customer_tariff/json/part-00000-44d37ba9-fccd-40f4-b939-d8b47af99c95-c000.json',
  '{"contract_id":"04_02_111_CHR39","target_local_start_timestamp":"2022-12-01T00:00:00.000Z","target_local_end_timestamp":"2023-02-16T00:00:00.000Z","tariff_name":"Electric Constant","charge_type":"buy","price":0.1166}\n{"contract_id":"04_02_111_CHR39","target_local_start_timestamp":"2023-02-16T00:00:00.000Z","target_local_end_timestamp":"2024-05-19T00:00:00.000Z","tariff_name":"Electric Constant","charge_type":"buy","price":0.1797}\n{"contract_id":"04_02_111_CHR39","target_local_start_timestamp":"2024-05-19T00:00:00.000Z","target_local_end_timestamp":"2024-08-31T00:00:00.000Z","tariff_name":"Eco Electric","charge_type":"buy","price":0.1911}\n{"contract_id":"04_02_111_CHR39","target_local_start_timestamp":"2024-08-31T00:00:00.000Z","target_local_end_timestamp":"2024-10-09T00:00:00.000Z","tariff_name":"Electric Constant","ch

In [5]:
# df_csv = spark.read.format('csv') \
#     .option('header', 'true') \
#     .load('project_data/project/raw_time_series/csv/part-00000-79a1db9d-9c38-4283-94cb-89c1f27d8d3f-c000.csv')
# df_csv.show()

df_json = spark.read.format('json') \
    .load('project_data/project/customer_tariff/json/part-00000-44d37ba9-fccd-40f4-b939-d8b47af99c95-c000.json')
df_json.show()

# df_parquet = spark.read.format('parquet') \
#     .load('project_data/project/raw_time_series/parquet/*')
# df_parquet.show()

+-----------+---------------+------+--------------------------+----------------------------+-----------------+
|charge_type|    contract_id| price|target_local_end_timestamp|target_local_start_timestamp|      tariff_name|
+-----------+---------------+------+--------------------------+----------------------------+-----------------+
|        buy|04_02_111_CHR39|0.1166|      2023-02-16T00:00:...|        2022-12-01T00:00:...|Electric Constant|
|        buy|04_02_111_CHR39|0.1797|      2024-05-19T00:00:...|        2023-02-16T00:00:...|Electric Constant|
|        buy|04_02_111_CHR39|0.1911|      2024-08-31T00:00:...|        2024-05-19T00:00:...|     Eco Electric|
|        buy|04_02_111_CHR39|0.1938|      2024-10-09T00:00:...|        2024-08-31T00:00:...|Electric Constant|
|        buy|04_02_111_CHR39| 0.201|      2024-11-09T00:00:...|        2024-10-09T00:00:...|     Eco Electric|
|        buy|04_02_111_CHR39|0.2447|      2024-12-11T00:00:...|        2024-11-09T00:00:...| Digital Electric|
|

In [13]:
from pyspark.sql import types as T

data_schema = T.StructType([
    T.StructField('annotations', T.StringType(), True),
    T.StructField('contract_id', T.StringType(), True),
    T.StructField('timestamp', T.StringType(), True),
    T.StructField('value', T.DoubleType(), True),
    T.StructField('value_source', T.StringType(), True)
])

df_raw_time_series = spark.read.format('json') \
    .schema(data_schema) \
    .load('project_data/project/raw_time_series/json/part-00000-3992c3d5-97c8-4e7d-8328-b7c8379135e8-c000.json')
df_raw_time_series.show()

+--------------------+--------------------+--------------------+-------------------+------------+
|         annotations|         contract_id|           timestamp|              value|value_source|
+--------------------+--------------------+--------------------+-------------------+------------+
|{"region":"Europe...|  04 _02 _111 _CHR12|2023-01-01T01:15:...|0.01887007418980177| measurement|
|{"region":"Europe...|  04_02 _ 111 _CHR12|2023-01-01T14:15:...|           1.266225| measurement|
|{"region":"Europe...|  04_02_ 111 _CHR12 |2023-01-01T21:45:...|               NULL| measurement|
|{"region":"Europe...|  04 _02 _111 _CHR12|2023-01-02T01:30:...|0.02093419915470171| measurement|
|{"region":"Europe...|   04 _02_111_CHR12 |2023-01-02T07:45:...| 0.0142526590154003| measurement|
|{"region":"Europe...|  04_02 _ 111 _CHR12|2023-01-02T16:45:...|0.07874463371047623| measurement|
|{"region":"Europe...|   04 _02_111_CHR12 |2023-01-02T22:45:...| 0.1950477376343327| measurement|
|{"region":"Europe..

In [14]:
data_schema = T.StructType([
    T.StructField('charge_type', T.StringType(), True),
    T.StructField('contract_id', T.StringType(), True),
    T.StructField('price', T.DoubleType(), True),
    T.StructField('target_local_start_timestamp', T.StringType(), True),
    T.StructField('target_local_end_timestamp', T.StringType(), True),
    T.StructField('tariff_name', T.StringType(), True)
])

df_raw_time_series = spark.read.format('json') \
    .schema(data_schema) \
    .load('project_data/project/customer_tariff/json/part-00000-44d37ba9-fccd-40f4-b939-d8b47af99c95-c000.json')
df_raw_time_series.show()

+-----------+---------------+------+----------------------------+--------------------------+-----------------+
|charge_type|    contract_id| price|target_local_start_timestamp|target_local_end_timestamp|      tariff_name|
+-----------+---------------+------+----------------------------+--------------------------+-----------------+
|        buy|04_02_111_CHR39|0.1166|        2022-12-01T00:00:...|      2023-02-16T00:00:...|Electric Constant|
|        buy|04_02_111_CHR39|0.1797|        2023-02-16T00:00:...|      2024-05-19T00:00:...|Electric Constant|
|        buy|04_02_111_CHR39|0.1911|        2024-05-19T00:00:...|      2024-08-31T00:00:...|     Eco Electric|
|        buy|04_02_111_CHR39|0.1938|        2024-08-31T00:00:...|      2024-10-09T00:00:...|Electric Constant|
|        buy|04_02_111_CHR39| 0.201|        2024-10-09T00:00:...|      2024-11-09T00:00:...|     Eco Electric|
|        buy|04_02_111_CHR39|0.2447|        2024-11-09T00:00:...|      2024-12-11T00:00:...| Digital Electric|
|